In [18]:
import numpy as np
import pandas as pd

In [1]:
import mlflow
mlflow.set_tracking_uri(uri="http://localhost:5000")

In [8]:
import lightgbm as lgb

lgbc = lgb.Booster(model_file='lightgbm/lgb.txt')

We used our saved lightgbm model 

In [24]:
np.argmax(lgbc.predict([[-0.657196,-2.27162,1.324874,-0.097875,3.637970,-3.413761,0.790723217]]),axis=1).item()

1

In [32]:
mlflow.set_experiment('2_model_registry')
with mlflow.start_run():
    mlflow.log_params(lgbc.params)
    mlflow.set_tag("Model","lightgbm")

    mlflow.sklearn.log_model('lgbc',artifact_path='models')
    mlflow.set_tag("Prediction",np.argmax(lgbc.predict([[-0.657196,-2.27162,1.324874,-0.097875,3.637970,-3.413761,0.790723217]]),axis=1).item())

2024/01/31 13:58:39 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


/home/ubuntu/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/ubuntu/miniconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [33]:
from mlflow.tracking import MlflowClient

In [36]:
client= MlflowClient(tracking_uri=mlflow.get_tracking_uri())

In [43]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1706708938289, experiment_id='3', last_update_time=1706708938289, lifecycle_stage='active', name='2_model_registry', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/2', creation_time=1705592485729, experiment_id='2', last_update_time=1705592485729, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/notebooks/mlruns/1', creation_time=1705591891060, experiment_id='1', last_update_time=1705591891060, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1705588661834, experiment_id='0', last_update_time=1705588661834, lifecycle_stage='active', name='Default', tags={}>]

In [44]:
client.create_experiment("lightgbm prediction experiment")

'4'

In [62]:
# lets check the versions
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids=3,
    #filter_string="metrics.rmse < 7",
    ##max_results=5,
    #order_by=["metrics.rmse ASC"]
)

In [75]:
for run in runs:
    print(f"run id:{run.info.run_id},tags: {run.data.tags}")

run id:66ab4427e1e041468c321b4083bcae69,tags: {'mlflow.user': 'ubuntu', 'mlflow.source.name': '/home/ubuntu/miniconda3/lib/python3.11/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.runName': 'sassy-cub-23', 'Model': 'lightgbm', 'mlflow.log-model.history': '[{"run_id": "66ab4427e1e041468c321b4083bcae69", "artifact_path": "models", "utc_time_created": "2024-01-31 13:58:39.874694", "flavors": {"sklearn": {"pickled_model": "model.pkl", "sklearn_version": "1.3.2", "serialization_format": "cloudpickle", "code": null}}, "model_uuid": "f6f20efbebda4aa683c56858ead8586c", "mlflow_version": "2.9.2", "model_size_bytes": 19}]', 'Prediction': '1'}
run id:3a9aa366f3be46bf9205f237781b64c4,tags: {'mlflow.user': 'ubuntu', 'mlflow.source.name': '/home/ubuntu/miniconda3/lib/python3.11/site-packages/ipykernel_launcher.py', 'mlflow.source.type': 'LOCAL', 'mlflow.runName': 'able-steed-395', 'Model': 'lightgbm', 'mlflow.log-model.history': '[{"run_id": "3a9aa366f3be46bf9205f23778

In [77]:
# interacting with the models registry

run_id = 'b8360bc5d0794087a24da537c19a352d'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="first_prediction_model")

Registered model 'first_prediction_model' already exists. Creating a new version of this model...
2024/01/31 14:21:08 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: first_prediction_model, version 1
Created version '1' of model 'first_prediction_model'.


<ModelVersion: aliases=[], creation_timestamp=1706710868977, current_stage='None', description='', last_updated_timestamp=1706710868977, name='first_prediction_model', run_id='b8360bc5d0794087a24da537c19a352d', run_link='', source='mlflow-artifacts:/3/b8360bc5d0794087a24da537c19a352d/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>

In [79]:
model_name = "first_prediction_model"
new_stage = "staging"
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage=new_stage,
    archive_existing_versions=True
)

/tmp/ipykernel_2112/3446462618.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=['new', 'production'], creation_timestamp=1706710868977, current_stage='Staging', description='', last_updated_timestamp=1706711119036, name='first_prediction_model', run_id='b8360bc5d0794087a24da537c19a352d', run_link='', source='mlflow-artifacts:/3/b8360bc5d0794087a24da537c19a352d/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>